In [ ]:
## IMPORTING FILES
import pandas as pd
import glob
import os
import numpy as np

# Find all CSVs in the datasets folder
files = glob.glob("../Datasets/*.csv")
print("FILES FOUND:", files)

df_list = []

for f in files:
    # Read the month's CSV
    temp = pd.read_csv(f)
    
    # Get month name from filename: "datasets/January.csv" -> "January"
    month = os.path.basename(f).replace(".csv", "")
    
    # Add a Month column
    temp["Month"] = month
    
    # Add to list
    df_list.append(temp)
print(df_list)
# Stack all months into one DataFrame
df = pd.concat(df_list, ignore_index=True)
print("Combined shape:", df.shape)
print("Months in data:", df["Month"].unique())
print(df.info())


In [45]:
## CLEANING COLUMNS AND NAMES
pd.set_option('display.max_columns', None)
print(df.info())
currency_cols = [
    "Room Rate in USD",
    "High Season 1 Room Rate (Sakura Season) (Mid Mar-End Apr) April 5-11",
    "High Season 2 Room Rate (Mid Oct-End Nov) (I used Nov 1-2 as the date here - Nick)",
    "Low Season 1 Room Rate (Jun-Mid July) June 21-27",
    "Low Season 2 Room Rate (Mid Jan-Feb) (I used Feb 1-2 as the date here)",
    "Avg High Season Rate",
    "Avg Low Season Rate",
    "ADR_Month (Low Season Rate + (High Season Rate - Low Season Rate) * Seasonality index",
    "RevPAR_month (ADR_month*Occupancy Rate)",
    "Revenue per night (Avg room rate * room count)"
]

def clean_currency(col):
    return (
        col.astype(str)
           .str.replace("$", "", regex=False)
           .str.replace(",", "", regex=False)
           .astype(float)
    )

for col in currency_cols:
    if col in df.columns:
        df[col] = clean_currency(df[col])
        
df.rename(columns={"Room Rate in USD": "Room Rate", "High Season 1 Room Rate (Sakura Season) (Mid Mar-End Apr) April 5-11": "High Season 1 Room Rate", "High Season 2 Room Rate (Mid Oct-End Nov) (I used Nov 1-2 as the date here - Nick)": "High Season 2 Room Rate",  "Low Season 1 Room Rate (Jun-Mid July) June 21-27": "Low Season 1 Room Rate", "Low Season 2 Room Rate (Mid Jan-Feb) (I used Feb 1-2 as the date here)": "Low Season 2 Room Rate", "ADR_Month (Low Season Rate + (High Season Rate - Low Season Rate) * Seasonality index": "ADR_Month", "RevPAR_month (ADR_month*Occupancy Rate)": "RevPAR_month", "Revenue per night (Avg room rate * room count)": "Revenue per night", "Distance to Closest Station (km)": "Distance to Closest Station", "Property Type (Star Rating)": "Property Type"}, inplace=True)


# 1. Find all occupancy columns
print("All columns:", df.columns.tolist())  # optional, but useful while debugging

occupancy_cols = [c for c in df.columns if "Occupancy Rate" in c]
print("Occupancy columns found:", occupancy_cols)

if len(occupancy_cols) == 0:
    raise ValueError("No occupancy columns found. Check the exact column names above.")

# 2. For each row, take the first non-null occupancy value
occ_matrix = df[occupancy_cols]

# sanity check shape
print("occ_matrix shape:", occ_matrix.shape)

df["Occupancy_Rate"] = occ_matrix.bfill(axis=1).iloc[:, 0]

print("Missing Occupancy_Rate:", df["Occupancy_Rate"].isna().sum())

# 3. Drop original occupancy columns
df = df.drop(columns=occupancy_cols)

# --- Drop scaled / redundant columns your team decided on ---

drop_cols = [
    "Rating Scaled",
    "Competitor density scaled",
    "Property Type Scaled",
    "Distance to Closest Station Scaled",
    "Walking Minutes",
    "Room Count",
    "High Season 1 Room Rate",
    "High Season 2 Room Rate",
    "Low Season 1 Room Rate",
    "Low Season 2 Room Rate"
]

df = df.drop(columns=[c for c in drop_cols if c in df.columns])
print(df.info())
print(df[["Hotel Name", "Month", "Occupancy_Rate"]].head(15))
print(df["Month"].value_counts())

df.to_csv("cleaned_hotels_all_months.csv", index=False)
print("Saved cleaned_hotels_all_months.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 35 columns):
 #   Column                                                                                 Non-Null Count  Dtype  
---  ------                                                                                 --------------  -----  
 0   Hotel Name                                                                             600 non-null    object 
 1   Coordinates                                                                            600 non-null    object 
 2   Room Count                                                                             600 non-null    int64  
 3   Rating                                                                                 600 non-null    float64
 4   Rating Scaled                                                                          600 non-null    float64
 5   Room Rate in USD                                                              